# Seção 3: Otimização do Tempo de Execução

___

# Imports  Para a Aula

In [1]:
import numpy as np

In [2]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier

In [3]:
from tree import BinaryTree, extract_tree_from_model, print_tree

___

# Tutorial

## Motivação: Contagem Simples

### Criando dados aleatórios

In [4]:
np.random.seed(123456789)
X = np.random.randint(10, 35, int(1e7))
X.shape, X[:10]

((10000000,), array([34, 28, 22, 17, 14, 25, 29, 23, 30, 10]))

### Função "Mágica" _%time_

In [5]:
%%time
count1 = 0
for x in X:
    if x > 20:
        count1 += 1

Wall time: 3.23 s


In [6]:
%%time
count2 = (X > 20).sum()

Wall time: 29.1 ms


In [7]:
assert count1 == count2

### Função "Mágica" _%timeit_

In [8]:
%%timeit
count1 = 0
for x in X:
    if x > 20:
        count1 += 1

2.03 s ± 199 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
%%timeit
count2 = (X > 20).sum()

42 ms ± 5.47 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [10]:
assert count1 == count2

## Motivação: Transformação Linear

### Funções Auxiliares

In [11]:
def matrix_dot_product(m1, m2):    
    assert (m1.shape[1] == m2.shape[0])
    P, Q = m1.shape
    Q, R = m2.shape
    ans = np.zeros((P, R))
    for p in range(P):
        for r in range(R):
            for q in range(Q):
                ans[p, r] += m1[p, q] * m2[q, r]
    return ans

In [12]:
def matrix_add(m1, m2):
    assert (m1.shape == m2.shape)
    P, Q = m1.shape
    ans = np.zeros((P, Q))
    for p in range(P):
        for q in range(Q):
            ans[p, q] += m1[p, q] + m2[p, q]
    return ans

### Criando dados aleatórios

In [13]:
np.random.seed(123456789)
P = 500
Q = 50
R = 1
A = np.random.randint(-10, 35, (P, Q))
X = np.random.randn(Q, R) * 10 + 3
B = np.random.randn(P, R) * 3 + 10
A.shape, X.shape, B.shape

((500, 50), (50, 1), (500, 1))

### Função "Mágica" _%time_

In [14]:
%%time
Y1 = matrix_add(matrix_dot_product(A, X), B)

Wall time: 143 ms


In [15]:
%%time
Y2 = np.add(np.dot(A, X), B)

Wall time: 67.2 ms


In [16]:
assert sum((Y1 - Y2) ** 2) ** .5 < 1e-10

### Função "Mágica" _%timeit_

In [17]:
%%timeit
Y1 = matrix_add(matrix_dot_product(A, X), B)

69.2 ms ± 3.21 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [18]:
%%timeit
Y2 = np.add(np.dot(A, X), B)

28.7 µs ± 803 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [19]:
assert sum((Y1 - Y2) ** 2) ** .5 < 1e-10

___

# Desafio

## Objetivo:

Construir uma classe de Árvore de Decisão derivada de uma classe de Árvore Binária, disponibilizada aos alunos no arquivo ``tree.py`` deste módulo. 

Não é necessário implementar o **treinamento** (ou métoro **fit**) da Árvode de Decisão; tampouco o método de predição de probabilidade (**predict_proba**). Todos os parâmetros da Árvore de Decisão serão extraídos de um modelo de `DecisionTreeClassifier` treinado com o dataset `Iris`.

Essa classe deve extender a classe disponibilizada `BinaryTree`. Cada instância dessa classe representa **apenas um nó** de uma árvore binária; este nó está conectado aos outros nós através dos seguintes atributos:
- `parent`
- `left_child`
- `right_child`

A classe `BinaryTree` não possui um método `predict`; este deve ser implementado para funcionar da seguinte forma:
1. Se for um **nó folha**, retorna a predição para o **nó pai**;
2. Se for um **nó de decisão**, deve processar a entrada `X` e chamar o método `predict` do próximo nó filho.

O método `predict` deve receber um array `X` contendo uma massa de dados sobre os quais serão feitas as predições, que serão retornadas no array `y_pred`.

## Árvore de Decisão

### Treinando uma Árvore de Decisão para o Problema de Classificação Iris

Referências:
- [Problema de Classificação: Iris](https://en.wikipedia.org/wiki/Iris_flower_data_set)
- [Árvore de Decisão para Classificação](http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html)

In [20]:
iris = load_iris()

In [21]:
model = DecisionTreeClassifier(max_depth=3, random_state=123456789)

In [22]:
model.fit(X=iris['data'], y=iris['target'])

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False,
            random_state=123456789, splitter='best')

### Testando a Predição da Árvore de Decisão Treinada

In [23]:
""" predições """
y_pred = model.predict(iris['data'])

In [24]:
""" targets / tabela verdade """
y_true = iris['target']

In [25]:
""" Acurácia do Modelo """
(y_pred == y_true).mean()

0.97333333333333338

### Parâmetros da Árvore de Decisão Treinada:

In [26]:
tree = extract_tree_from_model(model, BinaryTree)
print_tree(tree)

ID: 0 - Value: {'feature': 3, 'threshold': 0.80000001192092896} - Root Node - Children (2): {'L': 1, 'R': 2}
	ID: 1 - Value: {'probability': 1.0, 'decision': 0} - Parent: 0 - Leaf Node
	ID: 2 - Value: {'feature': 3, 'threshold': 1.75} - Parent: 0 - Children (2): {'L': 3, 'R': 6}
		ID: 3 - Value: {'feature': 2, 'threshold': 4.9499998092651367} - Parent: 2 - Children (2): {'L': 4, 'R': 5}
			ID: 4 - Value: {'probability': 0.97916666666666663, 'decision': 1} - Parent: 3 - Leaf Node
			ID: 5 - Value: {'probability': 0.66666666666666663, 'decision': 2} - Parent: 3 - Leaf Node
		ID: 6 - Value: {'feature': 2, 'threshold': 4.8500003814697266} - Parent: 2 - Children (2): {'L': 7, 'R': 8}
			ID: 7 - Value: {'probability': 0.66666666666666663, 'decision': 2} - Parent: 6 - Leaf Node
			ID: 8 - Value: {'probability': 1.0, 'decision': 2} - Parent: 6 - Leaf Node


#### Nota Importante:

Algumas definições deveriam ter sido dadas para melhor compreensão de como fazer a solução. São elas:

1. Um nó é um **nó folha** caso `self.value` seja um dicionário contendo o campo `decision`; o valor contido é a decisão de a qual classe pertence X.
2. O outro tipo de nó, que decide para onde X deve ir, contém 2 valores em `self.value`:
    - `feature`: qual coluna de X deve ser testada;
    - `threshold`: valor para o qual:
        - 1) `X[feature] <= threshold`, então deve-se seguir para a sub-àrvore da esquerda
        - 2) `X[feature] > threshold`, então deve-se seguir para a sub-àrvore da direita
        
Em um nó, se este for **folha**, deve-se retornar `y`, um `np.array` de mesma dimensão que `X.shape[0]` contendo como valor a `decisão`.

Se o nó for do outro tipo, deve-se testar cada elemento de `X` para enviar para a sub-árvore esquerda ou direita; o valor retornado por cada `X` deve ser colocado em `y` na ordem correta.

## Parte 1: Implementar a solução usando apenas loops

### Solução:

In [27]:
class LoopDecisionTree(BinaryTree):
    
    def __init__(self, threshold=None, feature=None, decision=None, *args, **kwargs):
        super(LoopDecisionTree, self).__init__(*args, **kwargs)
        if decision is None:
            if None in [threshold, feature]:
                raise Exception("Decision Tree (init): missing 'threshold' or 'feature' in non-leaf node.")    
            self.value = "X[{}] <= {}".format(feature, threshold)
        else:
            self.value = "Decision: {}".format(decision)
        self.decision = decision
        self.threshold = threshold
        self.feature = feature
        
    
    def predict(self, X):   
        if self.is_leaf():
            return np.array([self.decision] * len(X))
        y = []
        for x in X:
            if x[self.feature] <= self.threshold:
                y.append(self.left_child.predict([x])[0])
            else:
                y.append(self.right_child.predict([x])[0])
        return np.array(y)
            

### Avaliação da Solução

In [28]:
""" Construindo a árvore de decisão """
# classe de Árvore de Decisão
DTClass = LoopDecisionTree

# Criando os Nós da Árvore
n0 = DTClass(id=0, feature=3, threshold=0.80000001192092896)
n1 = DTClass(id=1, decision=0)
n2 = DTClass(id=2, feature=3, threshold=1.75)
n3 = DTClass(id=3, feature=2, threshold=4.9499998092651367)
n4 = DTClass(id=4, decision=1)
n5 = DTClass(id=5, decision=2)
n6 = DTClass(id=6, feature=2, threshold=4.8500003814697266)
n7 = DTClass(id=7, decision=2)
n8 = DTClass(id=8, decision=2)

# Construindo a Árvore
n0.append_left_child(n1)
n0.append_right_child(n2)
n2.append_left_child(n3)
n2.append_right_child(n6)
n3.append_left_child(n4)
n3.append_right_child(n5)
n6.append_left_child(n7)
n6.append_right_child(n8)

loop_decision_tree = n0

print_tree(loop_decision_tree)

ID: 0 - Value: X[3] <= 0.800000011920929 - Root Node - Children (2): {'L': 1, 'R': 2}
	ID: 1 - Value: Decision: 0 - Parent: 0 - Leaf Node
	ID: 2 - Value: X[3] <= 1.75 - Parent: 0 - Children (2): {'L': 3, 'R': 6}
		ID: 3 - Value: X[2] <= 4.949999809265137 - Parent: 2 - Children (2): {'L': 4, 'R': 5}
			ID: 4 - Value: Decision: 1 - Parent: 3 - Leaf Node
			ID: 5 - Value: Decision: 2 - Parent: 3 - Leaf Node
		ID: 6 - Value: X[2] <= 4.850000381469727 - Parent: 2 - Children (2): {'L': 7, 'R': 8}
			ID: 7 - Value: Decision: 2 - Parent: 6 - Leaf Node
			ID: 8 - Value: Decision: 2 - Parent: 6 - Leaf Node


In [29]:
assert (model.predict(iris['data']) == loop_decision_tree.predict(iris['data'])).all()

## Parte 2: Implementar a solução usando _numpy_

### Solução:

In [30]:
""" Escreva a a Solução Aqui """
class VectorDecisionTree(BinaryTree):    
        
    def __init__(self, threshold=None, feature=None, decision=None, *args, **kwargs):
        super(VectorDecisionTree, self).__init__(*args, **kwargs)
        if decision is None:
            if None in [threshold, feature]:
                raise Exception("Decision Tree (init): missing 'threshold' or 'feature' in non-leaf node.")    
            self.value = "X[{}] <= {}".format(feature, threshold)
        else:
            self.value = "Decision: {}".format(decision)
        self.decision = decision
        self.threshold = threshold
        self.feature = feature
        
    
    def predict(self, X):   
        if self.is_leaf():
            return np.ones(X.shape[0]) * self.decision
        y = np.zeros(X.shape[0])
        index = X[:, self.feature] <= self.threshold
        y[index] = self.left_child.predict(X[index])
        y[~index] = self.right_child.predict(X[~index])
        return y

### Avaliação da Solução

In [31]:
""" Construindo a árvore de decisão """
# classe de Árvore de Decisão
DTClass = VectorDecisionTree

# Criando os Nós da Árvore
n0 = DTClass(id=0, feature=3, threshold=0.80000001192092896)
n1 = DTClass(id=1, decision=0)
n2 = DTClass(id=2, feature=3, threshold=1.75)
n3 = DTClass(id=3, feature=2, threshold=4.9499998092651367)
n4 = DTClass(id=4, decision=1)
n5 = DTClass(id=5, decision=2)
n6 = DTClass(id=6, feature=2, threshold=4.8500003814697266)
n7 = DTClass(id=7, decision=2)
n8 = DTClass(id=8, decision=2)

# Construindo a Árvore
n0.append_left_child(n1)
n0.append_right_child(n2)
n2.append_left_child(n3)
n2.append_right_child(n6)
n3.append_left_child(n4)
n3.append_right_child(n5)
n6.append_left_child(n7)
n6.append_right_child(n8)

vector_decision_tree = n0

print_tree(vector_decision_tree)

ID: 0 - Value: X[3] <= 0.800000011920929 - Root Node - Children (2): {'L': 1, 'R': 2}
	ID: 1 - Value: Decision: 0 - Parent: 0 - Leaf Node
	ID: 2 - Value: X[3] <= 1.75 - Parent: 0 - Children (2): {'L': 3, 'R': 6}
		ID: 3 - Value: X[2] <= 4.949999809265137 - Parent: 2 - Children (2): {'L': 4, 'R': 5}
			ID: 4 - Value: Decision: 1 - Parent: 3 - Leaf Node
			ID: 5 - Value: Decision: 2 - Parent: 3 - Leaf Node
		ID: 6 - Value: X[2] <= 4.850000381469727 - Parent: 2 - Children (2): {'L': 7, 'R': 8}
			ID: 7 - Value: Decision: 2 - Parent: 6 - Leaf Node
			ID: 8 - Value: Decision: 2 - Parent: 6 - Leaf Node


In [32]:
assert (model.predict(iris['data']) == vector_decision_tree.predict(iris['data'])).all()

## Parte 3: Avaliar a diferença de velocidade

### Solução:

In [33]:
%%timeit
loop_decision_tree.predict(iris['data'])

859 µs ± 111 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [34]:
%%timeit
vector_decision_tree.predict(iris['data'])

72.7 µs ± 1.18 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
